# PRD Generation Agent for SolvIQ Project

This notebook creates an AI agent that can generate comprehensive Product Requirement Documents (PRDs) specifically tailored for the SolvIQ Project, which can be used as Cursor project rules (`.cursor/rules/new-prd.mdc`).

## Overview

The PRD Generation Agent will:
1. **Analyze project requirements** based on SolvIQ's AIE Certification challenge context
2. **Generate structured PRDs** with proper sections and formatting
3. **Create Cursor-compatible rules** for development workflow
4. **Provide templates** for different types of features and components

## SolvIQ Project Context

SolvIQ is an AI-powered problem-solving platform designed for the AIE Certification challenge. It focuses on:
- **Intelligent problem analysis**
- **Multi-step solution generation**
- **Interactive learning experiences**
- **Progress tracking and assessment**


## Setup and Imports


In [8]:
import os
import getpass
from datetime import datetime
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field
from agents import Agent, Runner
import nest_asyncio

# Enable nested async
nest_asyncio.apply()

# Set up API key
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")


## PRD Data Models

We'll define comprehensive data models for PRD generation that capture all necessary information for SolvIQ development.


In [9]:
class UserStory(BaseModel):
    """Individual user story with acceptance criteria"""
    title: str = Field(description="Brief title of the user story")
    description: str = Field(description="As a [user type], I want [goal] so that [benefit]")
    acceptance_criteria: List[str] = Field(description="List of acceptance criteria")
    priority: str = Field(description="High/Medium/Low priority")
    story_points: int = Field(description="Estimated story points (1-8)")

class TechnicalRequirement(BaseModel):
    """Technical requirements and constraints"""
    component: str = Field(description="Component or system name")
    requirements: List[str] = Field(description="Technical requirements")
    constraints: List[str] = Field(description="Technical constraints")
    dependencies: List[str] = Field(description="External dependencies")
    performance_metrics: List[str] = Field(description="Performance requirements")

class SolvIQPRD(BaseModel):
    """Complete PRD structure for SolvIQ project"""
    # Basic Information
    title: str = Field(description="PRD title")
    version: str = Field(description="PRD version")
    created_date: str = Field(description="Creation date")
    last_updated: str = Field(description="Last update date")
    
    # Project Context
    project_overview: str = Field(description="High-level project description")
    business_objectives: List[str] = Field(description="Business objectives")
    target_audience: str = Field(description="Primary target audience")
    
    # Requirements
    user_stories: List[UserStory] = Field(description="User stories")
    technical_requirements: List[TechnicalRequirement] = Field(description="Technical requirements")
    
    # Design and Architecture
    system_architecture: str = Field(description="High-level system architecture")
    user_flow: str = Field(description="Primary user flow description")
    
    # Success Metrics
    success_metrics: List[str] = Field(description="Key success metrics")
    
    # Cursor Rules
    cursor_rules: str = Field(description="Formatted Cursor project rules")
    
    # Additional Context
    assumptions: List[str] = Field(description="Project assumptions")
    risks: List[str] = Field(description="Identified risks")
    timeline: str = Field(description="Development timeline")
    
    # AIE Certification Context
    aie_certification_goals: List[str] = Field(description="AIE certification specific goals")
    demo_day_requirements: List[str] = Field(description="Demo day specific requirements")


## SolvIQ Project Context and PRD Generation Agent


In [10]:
# SolvIQ Project Context
SOLVIQ_CONTEXT = {
    "project_name": "SolvIQ",
    "project_description": "An AI-powered problem-solving platform designed to help users tackle complex challenges through intelligent analysis, multi-step solution generation, and interactive learning experiences.",
    "target_audience": "Students, professionals, and lifelong learners seeking to improve their problem-solving skills",
    "business_objectives": [
        "Demonstrate advanced AI capabilities for the AIE Certification challenge",
        "Create an engaging problem-solving platform",
        "Showcase multi-modal AI interactions",
        "Provide measurable learning outcomes",
        "Deliver a compelling demo day presentation"
    ],
    "key_features": [
        "Intelligent problem analysis and breakdown",
        "Multi-step solution generation with explanations",
        "Interactive problem-solving sessions",
        "Progress tracking and skill assessment",
        "Adaptive learning recommendations",
        "Real-time feedback and hints"
    ],
    "tech_stack": [
        "OpenAI GPT models for problem analysis",
        "Vector databases for knowledge storage",
        "React/Next.js for frontend",
        "Python/FastAPI for backend",
        "PostgreSQL for data persistence",
        "Redis for caching and sessions"
    ],
    "aie_certification_focus": [
        "Advanced AI agent orchestration",
        "Multi-modal AI interactions",
        "Real-time AI decision making",
        "Scalable AI architecture",
        "AI safety and reliability"
    ]
}

# PRD Generation Prompt
PRD_GENERATION_PROMPT = """
You are a senior product manager and technical architect specializing in AI-powered educational platforms. 
You are tasked with creating comprehensive Product Requirement Documents (PRDs) for SolvIQ, an AI-powered 
problem-solving platform designed for the AIE Certification challenge.

## Your Role:
- Senior Product Manager with expertise in AI/ML products
- Technical architect with experience in scalable systems
- Educational technology specialist
- AIE Certification program expert

## SolvIQ Project Context:
SolvIQ is an AI-powered problem-solving platform that helps users tackle complex challenges through:
- Intelligent problem analysis and breakdown
- Multi-step solution generation with explanations
- Interactive problem-solving sessions
- Progress tracking and skill assessment
- Adaptive learning recommendations
- Real-time feedback and hints

## Your Task:
Given a specific feature or component request, create a comprehensive PRD that includes:

1. **Project Overview**: Clear description of the feature/component
2. **Business Objectives**: How this aligns with SolvIQ's goals and AIE certification
3. **User Stories**: Detailed user stories with acceptance criteria
4. **Technical Requirements**: Architecture, performance, and technical constraints
5. **System Architecture**: High-level technical design
6. **Success Metrics**: Measurable outcomes
7. **Cursor Rules**: Specific development guidelines for this component
8. **AIE Certification Focus**: How this demonstrates advanced AI capabilities
9. **Timeline**: Realistic development timeline
10. **Risks and Assumptions**: Identified risks and project assumptions

## Guidelines:
- Focus on AI/ML capabilities and advanced agent orchestration
- Ensure scalability and performance requirements
- Include proper error handling and safety measures
- Consider demo day presentation requirements
- Align with AIE certification learning objectives
- Provide actionable, specific requirements
- Include technical implementation details
- Consider user experience and accessibility

## Output Format:
Generate a comprehensive PRD that can be used as Cursor project rules (.cursor/rules/new-prd.mdc)
and provides clear guidance for development teams.
"""

# Create the PRD Generation Agent
prd_agent = Agent(
    name="PRDGenerationAgent",
    instructions=PRD_GENERATION_PROMPT,
    model="gpt-4o",  # Using GPT-4o for comprehensive analysis
    output_type=SolvIQPRD,
)


## PRD Generation Utility Functions


In [24]:
class PRDGenerator:
    """Utility class for generating and formatting PRDs"""
    
    def __init__(self):
        self.agent = prd_agent
        self.context = SOLVIQ_CONTEXT
    
    async def generate_prd(self, feature_request: str, component_type: str = "feature") -> SolvIQPRD:
        """Generate a comprehensive PRD for a given feature request"""
        
        # Create detailed input for the agent
        input_prompt = f"""
        Generate a comprehensive PRD for the following {component_type} request:
        
        **Feature/Component Request:**
        {feature_request}
        
        **Project Context:**
        Project Name: {self.context['project_name']}
        Description: {self.context['project_description']}
        Target Audience: {self.context['target_audience']}
        
        **Key Features:**
        {chr(10).join(f'- {feature}' for feature in self.context['key_features'])}
        
        **Tech Stack:**
        {chr(10).join(f'- {tech}' for tech in self.context['tech_stack'])}
        
        **AIE Certification Focus:**
        {chr(10).join(f'- {goal}' for goal in self.context['aie_certification_focus'])}
        
        Please create a comprehensive PRD that includes all required sections and can be used 
        as Cursor project rules for development.
        """
        
        # Run the agent
        result = await Runner.run(self.agent, input_prompt)
        return result.final_output_as(SolvIQPRD)
    
    def format_prd_for_cursor(self, prd: SolvIQPRD) -> str:
        """Format PRD as Cursor-compatible markdown"""
        
        cursor_content = f"""
# {prd.title}

**Version:** {prd.version}  
**Created:** {prd.created_date}  
**Last Updated:** {prd.last_updated}

## Project Overview

{prd.project_overview}

## Business Objectives

{chr(10).join(f'- {obj}' for obj in prd.business_objectives)}

## Target Audience

{prd.target_audience}

## User Stories

{self._format_user_stories(prd.user_stories)}

## Technical Requirements

{self._format_technical_requirements(prd.technical_requirements)}

## System Architecture

{prd.system_architecture}

## User Flow

{prd.user_flow}

## Success Metrics

{chr(10).join(f'- {metric}' for metric in prd.success_metrics)}

## AIE Certification Goals

{chr(10).join(f'- {goal}' for goal in prd.aie_certification_goals)}

## Demo Day Requirements

{chr(10).join(f'- {req}' for req in prd.demo_day_requirements)}

## Development Timeline

{prd.timeline}

## Assumptions

{chr(10).join(f'- {assumption}' for assumption in prd.assumptions)}

## Identified Risks

{chr(10).join(f'- {risk}' for risk in prd.risks)}

## Cursor Development Rules

{prd.cursor_rules}

---

*This PRD was generated by the SolvIQ PRD Generation Agent for AIE Certification challenge.*
"""
        
        return cursor_content.strip()
    
    def _format_user_stories(self, user_stories: List[UserStory]) -> str:
        """Format user stories section"""
        formatted_stories = []
        for i, story in enumerate(user_stories, 1):
            story_text = f"""
### {i}. {story.title}

**Description:** {story.description}

**Priority:** {story.priority}  
**Story Points:** {story.story_points}

**Acceptance Criteria:**
{chr(10).join(f'- {criteria}' for criteria in story.acceptance_criteria)}
"""
            formatted_stories.append(story_text.strip())
        
        return "\n\n".join(formatted_stories)
    
    def _format_technical_requirements(self, tech_reqs: List[TechnicalRequirement]) -> str:
        """Format technical requirements section"""
        formatted_reqs = []
        for req in tech_reqs:
            req_text = f"""
### {req.component}

**Requirements:**
{chr(10).join(f'- {r}' for r in req.requirements)}

**Constraints:**
{chr(10).join(f'- {c}' for c in req.constraints)}

**Dependencies:**
{chr(10).join(f'- {d}' for d in req.dependencies)}

**Performance Metrics:**
{chr(10).join(f'- {m}' for m in req.performance_metrics)}
"""
            formatted_reqs.append(req_text.strip())
        
        return "\n\n".join(formatted_reqs)
    
    async def save_prd_to_cursor_rules(self, prd: SolvIQPRD, filename: str = "new-prd.mdc"):
        """Save PRD as Cursor rules file"""
        cursor_content = self.format_prd_for_cursor(prd)
        
        # Create .cursor/rules directory if it doesn't exist
        import os
        os.makedirs(".cursor/rules", exist_ok=True)
        
        # Save the file
        with open(f".cursor/rules/{filename}", "w") as f:
            f.write(cursor_content)
        
        print(f"PRD saved to .cursor/rules/{filename}")
        return f".cursor/rules/{filename}"

# Initialize the PRD generator
prd_generator = PRDGenerator()


class PRDGenerator:
    """Utility class for generating and formatting PRDs"""
    
    def __init__(self):
        self.agent = prd_agent
        self.context = SOLVIQ_CONTEXT
    
    async def generate_prd(self, feature_request: str, component_type: str = "feature") -> SolvIQPRD:
        """Generate a comprehensive PRD for a given feature request"""
        
        # Create detailed input for the agent
        input_prompt = f"""
        Generate a comprehensive PRD for the following {component_type} request:
        
        **Feature/Component Request:**
        {feature_request}
        
        **Project Context:**
        Project Name: {self.context['project_name']}
        Description: {self.context['project_description']}
        Target Audience: {self.context['target_audience']}
        
        **Key Features:**
        {chr(10).join(f'- {feature}' for feature in self.context['key_features'])}
        
        **Tech Stack:**
        {chr(10).join(f'- {tech}' for tech in self.context['tech_stack'])}
        
        **AIE Certification Focus:**
        {chr(10).join(f'- {goal}' for goal in self.context['aie_certification_focus'])}
        
        Please create a comprehensive PRD that includes all required sections and can be used 
        as Cursor project rules for development.
        """
        
        # Run the agent
        result = await Runner.run(self.agent, input_prompt)
        return result.final_output_as(SolvIQPRD)
    
    def format_prd_for_cursor(self, prd: SolvIQPRD) -> str:
        """Format PRD as Cursor-compatible markdown"""
        
        cursor_content = f"""
# {prd.title}

**Version:** {prd.version}  
**Created:** {prd.created_date}  
**Last Updated:** {prd.last_updated}

## Project Overview

{prd.project_overview}

## Business Objectives

{chr(10).join(f'- {obj}' for obj in prd.business_objectives)}

## Target Audience

{prd.target_audience}

## User Stories

{self._format_user_stories(prd.user_stories)}

## Technical Requirements

{self._format_technical_requirements(prd.technical_requirements)}

## System Architecture

{prd.system_architecture}

## User Flow

{prd.user_flow}

## Success Metrics

{chr(10).join(f'- {metric}' for metric in prd.success_metrics)}

## AIE Certification Goals

{chr(10).join(f'- {goal}' for goal in prd.aie_certification_goals)}

## Demo Day Requirements

{chr(10).join(f'- {req}' for req in prd.demo_day_requirements)}

## Development Timeline

{prd.timeline}

## Assumptions

{chr(10).join(f'- {assumption}' for assumption in prd.assumptions)}

## Identified Risks

{chr(10).join(f'- {risk}' for risk in prd.risks)}

## Cursor Development Rules

{prd.cursor_rules}

---

*This PRD was generated by the SolvIQ PRD Generation Agent for AIE Certification challenge.*
"""
        
        return cursor_content.strip()
    
    def _format_user_stories(self, user_stories: List[UserStory]) -> str:
        """Format user stories section"""
        formatted_stories = []
        for i, story in enumerate(user_stories, 1):
            story_text = f"""
### {i}. {story.title}

**Description:** {story.description}

**Priority:** {story.priority}  
**Story Points:** {story.story_points}

**Acceptance Criteria:**
{chr(10).join(f'- {criteria}' for criteria in story.acceptance_criteria)}
"""
            formatted_stories.append(story_text.strip())
        
        return "\n\n".join(formatted_stories)
    
    def _format_technical_requirements(self, tech_reqs: List[TechnicalRequirement]) -> str:
        """Format technical requirements section"""
        formatted_reqs = []
        for req in tech_reqs:
            req_text = f"""
### {req.component}

**Requirements:**
{chr(10).join(f'- {r}' for r in req.requirements)}

**Constraints:**
{chr(10).join(f'- {c}' for c in req.constraints)}

**Dependencies:**
{chr(10).join(f'- {d}' for d in req.dependencies)}

**Performance Metrics:**
{chr(10).join(f'- {m}' for m in req.performance_metrics)}
"""
            formatted_reqs.append(req_text.strip())
        
        return "\n\n".join(formatted_reqs)
    
    async def save_prd_to_cursor_rules(self, prd: SolvIQPRD, filename: str = "new-prd.mdc"):
        """Save PRD as Cursor rules file"""
        cursor_content = self.format_prd_for_cursor(prd)
        
        # Create .cursor/rules directory if it doesn't exist
        import os
        os.makedirs(".cursor/rules", exist_ok=True)
        
        # Save the file
        with open(f".cursor/rules/{filename}", "w") as f:
            f.write(cursor_content)
        
        print(f"PRD saved to .cursor/rules/{filename}")
        return f".cursor/rules/{filename}"

# Initialize the PRD generator
prd_generator = PRDGenerator()


## Initialize PRD Generator

Before running examples, make sure all dependencies are loaded:


In [ ]:
# Initialize the PRD generator (run this cell first)
print("🔧 Initializing SolvIQ PRD Generation Agent...")

# Check if all required components are available
try:
    # Test if prd_agent is defined
    if 'prd_agent' not in globals():
        print("❌ prd_agent not found. Please run the previous cells first.")
    else:
        print("✅ PRD Agent initialized")
    
    # Test if SOLVIQ_CONTEXT is defined
    if 'SOLVIQ_CONTEXT' not in globals():
        print("❌ SOLVIQ_CONTEXT not found. Please run the previous cells first.")
    else:
        print("✅ SolvIQ Context loaded")
    
    # Initialize the generator
    prd_generator = PRDGenerator()
    print("✅ PRD Generator initialized")
    print("\n🚀 Ready to generate PRDs!")
    print("You can now run the example functions below.")
    
except Exception as e:
    print(f"❌ Error initializing: {e}")
    print("Please make sure you've run all the previous cells in order.")


🔧 Initializing SolvIQ PRD Generation Agent...
✅ PRD Agent initialized
✅ SolvIQ Context loaded
✅ PRD Generator initialized

🚀 Ready to generate PRDs!
You can now run the example functions below.


## Example Usage: Generate PRD for Problem Analysis Feature


In [ ]:
# Example: Generate PRD for Problem Analysis Feature
async def generate_problem_analysis_prd():
    """Generate PRD for the core problem analysis feature"""
    
    feature_request = """
    Create a comprehensive problem analysis system that can:
    
    1. Accept user-submitted problems in various formats (text, images, documents)
    2. Analyze and break down complex problems into manageable components
    3. Identify key concepts, constraints, and requirements
    4. Suggest problem-solving approaches and methodologies
    5. Provide real-time feedback and hints during analysis
    6. Store analysis results for future reference and learning
    
    This should be the core feature that demonstrates advanced AI capabilities 
    for the AIE Certification challenge and showcase intelligent problem 
    decomposition and analysis.
    """
    
    print("Generating PRD for Problem Analysis Feature...")
    print("This may take a moment as the agent analyzes requirements...")
    
    # Generate the PRD
    prd = await prd_generator.generate_prd(feature_request, "core feature")
    
    # Save to Cursor rules
    file_path = await prd_generator.save_prd_to_cursor_rules(prd, "problem-analysis-prd.mdc")
    
    print(f"\n✅ PRD generated successfully!")
    print(f"📁 Saved to: {file_path}")
    print(f"📋 Title: {prd.title}")
    print(f"📊 User Stories: {len(prd.user_stories)}")
    print(f"🔧 Technical Requirements: {len(prd.technical_requirements)}")
    
    return prd

# Uncomment the line below to run the example
problem_analysis_prd = await generate_problem_analysis_prd()


Generating PRD for Problem Analysis Feature...
This may take a moment as the agent analyzes requirements...
PRD saved to .cursor/rules/problem-analysis-prd.mdc

✅ PRD generated successfully!
📁 Saved to: .cursor/rules/problem-analysis-prd.mdc
📋 Title: SolvIQ Problem Analysis System PRD
📊 User Stories: 6
🔧 Technical Requirements: 3


## Interactive PRD Generation Interface

This PRD Generation Agent provides comprehensive tools for creating Product Requirement Documents specifically tailored for the SolvIQ project. Here's how to use it:


In [29]:
async def interactive_prd_generator():
    """Interactive PRD generation interface"""
    
    print("🚀 SolvIQ PRD Generation Agent")
    print("=" * 50)
    print("\nThis agent will help you create comprehensive PRDs for SolvIQ components.")
    print("\nAvailable component types:")
    print("1. Core Feature")
    print("2. AI/ML Component")
    print("3. User Interface")
    print("4. Backend Service")
    print("5. Integration")
    print("6. Custom")
    
    # Get user input
    component_type = input("\nSelect component type (1-6): ")
    
    type_mapping = {
        "1": "core feature",
        "2": "AI/ML component",
        "3": "user interface",
        "4": "backend service",
        "5": "integration",
        "6": "custom component"
    }
    
    selected_type = type_mapping.get(component_type, "custom component")
    
    print(f"\nSelected: {selected_type}")
    
    # Get feature description
    print("\nPlease describe the feature or component you want to create a PRD for:")
    print("(Be as detailed as possible - include goals, functionality, and any specific requirements)")
    
    feature_description = input("\nFeature Description: ")
    
    if not feature_description.strip():
        print("❌ No description provided. Exiting.")
        return
    
    # Generate filename
    filename = input("\nEnter filename for PRD (without extension): ").strip()
    if not filename:
        filename = f"{selected_type.replace(' ', '-')}-prd"
    
    filename = f"{filename}.mdc"
    
    print(f"\n🔄 Generating PRD for {selected_type}...")
    print("This may take a moment...")
    
    try:
        # Generate PRD
        prd = await prd_generator.generate_prd(feature_description, selected_type)
        
        # Save to file
        file_path = await prd_generator.save_prd_to_cursor_rules(prd, filename)
        
        print(f"\n✅ PRD generated successfully!")
        print(f"📁 Saved to: {file_path}")
        print(f"📋 Title: {prd.title}")
        print(f"📊 User Stories: {len(prd.user_stories)}")
        print(f"🔧 Technical Requirements: {len(prd.technical_requirements)}")
        print(f"🎯 Success Metrics: {len(prd.success_metrics)}")
        
        # Show preview
        print("\n📖 PRD Preview:")
        print("=" * 50)
        print(prd.project_overview[:200] + "..." if len(prd.project_overview) > 200 else prd.project_overview)
        
        print(f"\n🎉 Your PRD is ready! You can now use it as Cursor project rules.")
        
    except Exception as e:
        print(f"❌ Error generating PRD: {e}")
        print("Please try again with a different description.")

# Uncomment to run interactive generator
# await interactive_prd_generator()


In [30]:
# Quick PRD Generation Test
print("🚀 Testing PRD Generation...")

# Simple test to generate a basic PRD
async def quick_prd_test():
    try:
        feature_request = "Create a user authentication system for SolvIQ with login, registration, and password reset functionality."
        
        print("Generating PRD for User Authentication System...")
        prd = await prd_generator.generate_prd(feature_request, "authentication system")
        
        # Save to file
        file_path = await prd_generator.save_prd_to_cursor_rules(prd, "auth-system-prd.mdc")
        
        print(f"✅ PRD generated successfully!")
        print(f"📁 Saved to: {file_path}")
        print(f"📋 Title: {prd.title}")
        print(f"📊 User Stories: {len(prd.user_stories)}")
        
        return prd
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Run the test
test_prd = await quick_prd_test()


🚀 Testing PRD Generation...
Generating PRD for User Authentication System...
PRD saved to .cursor/rules/auth-system-prd.mdc
✅ PRD generated successfully!
📁 Saved to: .cursor/rules/auth-system-prd.mdc
📋 Title: SolvIQ Authentication System PRD
📊 User Stories: 3
